**MOUNT DRIVE**

In [ ]:
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
print("GPU Available: ", tf.test.is_gpu_available())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU Available:  False


*   CLAHE = /content/drive/MyDrive/Dataset224Clahe
*   Tanpa CLAHE = /content/drive/MyDrive/Dataset224




#**CLAHE (MENINGKATKAN KONTRAS DATASET) DAN RESIZE 224x224**

In [ ]:
import os
import cv2

input_folder = "/content/drive/MyDrive/Dataset"
output_folder = "/content/drive/MyDrive/Dataset224Clahe"

# Membuat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

categories = ["Normal", "Mass", "Nodule", "Pneumonia", "Tuberculosis"]

for category in categories:
    category_input_folder = os.path.join(input_folder, category)
    category_output_folder = os.path.join(output_folder, category)
    os.makedirs(category_output_folder, exist_ok=True)

    images = os.listdir(category_input_folder)

    for image_name in images:
        image_path = os.path.join(category_input_folder, image_name)
        output_image_path = os.path.join(category_output_folder, image_name)

        if os.path.isfile(image_path):
            # Baca gambar
            img = cv2.imread(image_path)

            # Normalisasi ukuran menjadi 224x224
            img = cv2.resize(img, (224, 224))

            # Lakukan penghapusan noise atau pemrosesan lainnya
            # Misalnya, menggunakan filter Gaussian untuk menghaluskan gambar
            
            # Gaussian
            # img = cv2.GaussianBlur(img, (5, 5), 0)

            # Ubah image ke grayscale
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Setting CLAHE
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            # Terapkan CLAHE pada gambar
            img_clahe = clahe.apply(img_gray)

            # Simpan gambar yang sudah dinormalisasi
            cv2.imwrite(output_image_path, img_clahe)

In [ ]:
import os

output_folder = "/content/drive/MyDrive/Dataset224Clahe"

categories = ["Normal", "Mass", "Nodule", "Pneumonia", "Tuberculosis"]

for category in categories:
    category_folder = os.path.join(output_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")

Jumlah file pada folder Normal: 2000
Jumlah file pada folder Mass: 2050
Jumlah file pada folder Nodule: 2000
Jumlah file pada folder Pneumonia: 2000
Jumlah file pada folder Tuberculosis: 2000


#**RESIZE 224x224 (TANPA CLAHE)**

In [ ]:
import os
import cv2

input_folder = "/content/drive/MyDrive/Dataset"
output_folder = "/content/drive/MyDrive/Dataset224"

# Membuat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

categories = ["Normal", "Mass", "Nodule", "Pneumonia", "Tuberculosis"]

for category in categories:
    category_input_folder = os.path.join(input_folder, category)
    category_output_folder = os.path.join(output_folder, category)
    os.makedirs(category_output_folder, exist_ok=True)

    images = os.listdir(category_input_folder)

    for image_name in images:
        image_path = os.path.join(category_input_folder, image_name)
        output_image_path = os.path.join(category_output_folder, image_name)

        if os.path.isfile(image_path):
            # Baca gambar
            img = cv2.imread(image_path)

            # Normalisasi ukuran menjadi 224x224
            img = cv2.resize(img, (224, 224))

            # Simpan gambar yang sudah dinormalisasi
            cv2.imwrite(output_image_path, img)

In [ ]:
import os

output_folder = "/content/drive/MyDrive/Dataset224"

categories = ["Normal", "Mass", "Nodule", "Pneumonia", "Tuberculosis"]

for category in categories:
    category_folder = os.path.join(output_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")

Jumlah file pada folder Normal: 2000
Jumlah file pada folder Mass: 2000
Jumlah file pada folder Nodule: 2000
Jumlah file pada folder Pneumonia: 2000
Jumlah file pada folder Tuberculosis: 2000


# ✈**SPLIT 90 TRAIN**


## **TRAIN 90 CLAHE**

In [ ]:
import os
import shutil
import random

# Tentukan path dataset
dataset_path = "/content/drive/MyDrive/Dataset224Clahe"

# Tentukan path output untuk menyimpan dataset terbagi
output_path = "/content/drive/MyDrive/DatasetSplit90Clahe"

# Buat direktori output jika belum ada
os.makedirs(output_path, exist_ok=True)

# Membaca nama file dari setiap kelas
normal_files = os.listdir(os.path.join(dataset_path, "Normal"))
pneumonia_files = os.listdir(os.path.join(dataset_path, "Pneumonia"))
mass_files = os.listdir(os.path.join(dataset_path, "Mass"))
nodule_files = os.listdir(os.path.join(dataset_path, "Nodule"))
tuberculosis_files = os.listdir(os.path.join(dataset_path, "Tuberculosis"))

# Menggabungkan semua kategori
all_files = {
    "Normal": normal_files,
    "Pneumonia": pneumonia_files,
    "Mass": mass_files,
    "Nodule": nodule_files,
    "Tuberculosis": tuberculosis_files
}

# Mengacak urutan file di setiap kategori
for category in all_files:
    random.shuffle(all_files[category])

# Tentukan rasio pembagian dataset
train_ratio = 0.9
validation_ratio = 0.05
test_ratio = 0.05

# Memisahkan dataset menjadi bagian pelatihan, validasi, dan pengujian
train_data = {}
validation_data = {}
test_data = {}

for category, files in all_files.items():
    total_files = len(files)
    train_split = int(train_ratio * total_files)
    validation_split = int(validation_ratio * total_files)

    train_data[category] = files[:train_split]
    validation_data[category] = files[train_split:train_split+validation_split]
    test_data[category] = files[train_split+validation_split:]

# Fungsi untuk memindahkan file ke direktori output
def move_files(files, source_path, destination_path):
    for file in files:
        source_file = os.path.join(source_path, file)
        destination_file = os.path.join(destination_path, file)
        shutil.copyfile(source_file, destination_file)

# Memindahkan file ke direktori output
for category, files in train_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "train", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in validation_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "validation", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in test_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "test", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

print("Split Data Telah Selesai!")

In [ ]:
import os

train_folder = "/content/drive/MyDrive/DatasetSplit90Clahe/train"
validation_folder = "/content/drive/MyDrive/DatasetSplit90Clahe/validation"
test_folder = "/content/drive/MyDrive/DatasetSplit90Clahe/test"

categories = ["Mass", "Nodule", "Normal", "Pneumonia", "Tuberculosis"]

print("TRAIN")
for category in categories:
    category_folder = os.path.join(train_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("VALIDATION")
for category in categories:
    category_folder = os.path.join(validation_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("TEST")
for category in categories:
    category_folder = os.path.join(test_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")

TRAIN
Jumlah file pada folder Mass: 1845
Jumlah file pada folder Nodule: 1800
Jumlah file pada folder Normal: 1800
Jumlah file pada folder Pneumonia: 1800
Jumlah file pada folder Tuberculosis: 1800
VALIDATION
Jumlah file pada folder Mass: 102
Jumlah file pada folder Nodule: 100
Jumlah file pada folder Normal: 100
Jumlah file pada folder Pneumonia: 100
Jumlah file pada folder Tuberculosis: 100
TEST
Jumlah file pada folder Mass: 103
Jumlah file pada folder Nodule: 100
Jumlah file pada folder Normal: 100
Jumlah file pada folder Pneumonia: 100
Jumlah file pada folder Tuberculosis: 100


## **TRAIN 90 TANPA CLAHE**

In [ ]:
import os
import shutil
import random

# Tentukan path dataset
dataset_path = "/content/drive/MyDrive/Dataset224"

# Tentukan path output untuk menyimpan dataset terbagi
output_path = "/content/drive/MyDrive/DatasetSplit90"

# Buat direktori output jika belum ada
os.makedirs(output_path, exist_ok=True)

# Membaca nama file dari setiap kelas
normal_files = os.listdir(os.path.join(dataset_path, "Normal"))
pneumonia_files = os.listdir(os.path.join(dataset_path, "Pneumonia"))
mass_files = os.listdir(os.path.join(dataset_path, "Mass"))
nodule_files = os.listdir(os.path.join(dataset_path, "Nodule"))
tuberculosis_files = os.listdir(os.path.join(dataset_path, "Tuberculosis"))

# Menggabungkan semua kategori
all_files = {
    "Normal": normal_files,
    "Pneumonia": pneumonia_files,
    "Mass": mass_files,
    "Nodule": nodule_files,
    "Tuberculosis": tuberculosis_files
}

# Mengacak urutan file di setiap kategori
for category in all_files:
    random.shuffle(all_files[category])

# Tentukan rasio pembagian dataset
train_ratio = 0.9
validation_ratio = 0.05
test_ratio = 0.05

# Memisahkan dataset menjadi bagian pelatihan, validasi, dan pengujian
train_data = {}
validation_data = {}
test_data = {}

for category, files in all_files.items():
    total_files = len(files)
    train_split = int(train_ratio * total_files)
    validation_split = int(validation_ratio * total_files)

    train_data[category] = files[:train_split]
    validation_data[category] = files[train_split:train_split+validation_split]
    test_data[category] = files[train_split+validation_split:]

# Fungsi untuk memindahkan file ke direktori output
def move_files(files, source_path, destination_path):
    for file in files:
        source_file = os.path.join(source_path, file)
        destination_file = os.path.join(destination_path, file)
        shutil.copyfile(source_file, destination_file)

# Memindahkan file ke direktori output
for category, files in train_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "train", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in validation_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "validation", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in test_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "test", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

print("Split Data Telah Selesai!")

Split Data Telah Selesai!


In [ ]:
import os

train_folder = "/content/drive/MyDrive/DatasetSplit90/train"
validation_folder = "/content/drive/MyDrive/DatasetSplit90/validation"
test_folder = "/content/drive/MyDrive/DatasetSplit90/test"

categories = ["Mass", "Nodule", "Normal", "Pneumonia", "Tuberculosis"]

print("TRAIN")
for category in categories:
    category_folder = os.path.join(train_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("VALIDATION")
for category in categories:
    category_folder = os.path.join(validation_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("TEST")
for category in categories:
    category_folder = os.path.join(test_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")

TRAIN
Jumlah file pada folder Mass: 1800
Jumlah file pada folder Nodule: 1800
Jumlah file pada folder Normal: 1800
Jumlah file pada folder Pneumonia: 1800
Jumlah file pada folder Tuberculosis: 1800
VALIDATION
Jumlah file pada folder Mass: 100
Jumlah file pada folder Nodule: 100
Jumlah file pada folder Normal: 100
Jumlah file pada folder Pneumonia: 100
Jumlah file pada folder Tuberculosis: 100
TEST
Jumlah file pada folder Mass: 100
Jumlah file pada folder Nodule: 100
Jumlah file pada folder Normal: 100
Jumlah file pada folder Pneumonia: 100
Jumlah file pada folder Tuberculosis: 100


# ✈**SPLIT 80 TRAIN**

## **TRAIN 80 CLAHE**

In [ ]:
import os
import shutil
import random

# Tentukan path dataset
dataset_path = "/content/drive/MyDrive/Dataset224Clahe"

# Tentukan path output untuk menyimpan dataset terbagi
output_path = "/content/drive/MyDrive/DatasetSplit80Clahe"

# Buat direktori output jika belum ada
os.makedirs(output_path, exist_ok=True)

# Membaca nama file dari setiap kelas
normal_files = os.listdir(os.path.join(dataset_path, "Normal"))
pneumonia_files = os.listdir(os.path.join(dataset_path, "Pneumonia"))
mass_files = os.listdir(os.path.join(dataset_path, "Mass"))
nodule_files = os.listdir(os.path.join(dataset_path, "Nodule"))
tuberculosis_files = os.listdir(os.path.join(dataset_path, "Tuberculosis"))

# Menggabungkan semua kategori
all_files = {
    "Normal": normal_files,
    "Pneumonia": pneumonia_files,
    "Mass": mass_files,
    "Nodule": nodule_files,
    "Tuberculosis": tuberculosis_files
}

# Mengacak urutan file di setiap kategori
for category in all_files:
    random.shuffle(all_files[category])

# Tentukan rasio pembagian dataset
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# Memisahkan dataset menjadi bagian pelatihan, validasi, dan pengujian
train_data = {}
validation_data = {}
test_data = {}

for category, files in all_files.items():
    total_files = len(files)
    train_split = int(train_ratio * total_files)
    validation_split = int(validation_ratio * total_files)

    train_data[category] = files[:train_split]
    validation_data[category] = files[train_split:train_split+validation_split]
    test_data[category] = files[train_split+validation_split:]

# Fungsi untuk memindahkan file ke direktori output
def move_files(files, source_path, destination_path):
    for file in files:
        source_file = os.path.join(source_path, file)
        destination_file = os.path.join(destination_path, file)
        shutil.copyfile(source_file, destination_file)

# Memindahkan file ke direktori output
for category, files in train_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "train", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in validation_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "validation", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in test_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "test", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

print("Split Data Telah Selesai!")

In [ ]:
import os

train_folder = "/content/drive/MyDrive/DatasetSplit80Clahe/train"
validation_folder = "/content/drive/MyDrive/DatasetSplit80Clahe/validation"
test_folder = "/content/drive/MyDrive/DatasetSplit80Clahe/test"

categories = ["Mass", "Nodule", "Normal", "Pneumonia", "Tuberculosis"]

print("TRAIN")
for category in categories:
    category_folder = os.path.join(train_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("VALIDATION")
for category in categories:
    category_folder = os.path.join(validation_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("TEST")
for category in categories:
    category_folder = os.path.join(test_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")

## **TRAIN 80 TANPA CLAHE**

In [ ]:
import os
import shutil
import random

# Tentukan path dataset
dataset_path = "/content/drive/MyDrive/Dataset224"

# Tentukan path output untuk menyimpan dataset terbagi
output_path = "/content/drive/MyDrive/DatasetSplit80"

# Buat direktori output jika belum ada
os.makedirs(output_path, exist_ok=True)

# Membaca nama file dari setiap kelas
normal_files = os.listdir(os.path.join(dataset_path, "Normal"))
pneumonia_files = os.listdir(os.path.join(dataset_path, "Pneumonia"))
mass_files = os.listdir(os.path.join(dataset_path, "Mass"))
nodule_files = os.listdir(os.path.join(dataset_path, "Nodule"))
tuberculosis_files = os.listdir(os.path.join(dataset_path, "Tuberculosis"))

# Menggabungkan semua kategori
all_files = {
    "Normal": normal_files,
    "Pneumonia": pneumonia_files,
    "Mass": mass_files,
    "Nodule": nodule_files,
    "Tuberculosis": tuberculosis_files
}

# Mengacak urutan file di setiap kategori
for category in all_files:
    random.shuffle(all_files[category])

# Tentukan rasio pembagian dataset
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# Memisahkan dataset menjadi bagian pelatihan, validasi, dan pengujian
train_data = {}
validation_data = {}
test_data = {}

for category, files in all_files.items():
    total_files = len(files)
    train_split = int(train_ratio * total_files)
    validation_split = int(validation_ratio * total_files)

    train_data[category] = files[:train_split]
    validation_data[category] = files[train_split:train_split+validation_split]
    test_data[category] = files[train_split+validation_split:]

# Fungsi untuk memindahkan file ke direktori output
def move_files(files, source_path, destination_path):
    for file in files:
        source_file = os.path.join(source_path, file)
        destination_file = os.path.join(destination_path, file)
        shutil.copyfile(source_file, destination_file)

# Memindahkan file ke direktori output
for category, files in train_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "train", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in validation_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "validation", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

for category, files in test_data.items():
    source_path = os.path.join(dataset_path, category)
    destination_path = os.path.join(output_path, "test", category)
    os.makedirs(destination_path, exist_ok=True)
    move_files(files, source_path, destination_path)

print("Split Data Telah Selesai!")

Split Data Telah Selesai!


In [ ]:
import os

train_folder = "/content/drive/MyDrive/DatasetSplit80/train"
validation_folder = "/content/drive/MyDrive/DatasetSplit80/validation"
test_folder = "/content/drive/MyDrive/DatasetSplit80/test"

categories = ["Mass", "Nodule", "Normal", "Pneumonia", "Tuberculosis"]

print("TRAIN")
for category in categories:
    category_folder = os.path.join(train_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("VALIDATION")
for category in categories:
    category_folder = os.path.join(validation_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")
print("TEST")
for category in categories:
    category_folder = os.path.join(test_folder, category)
    file_count = len(os.listdir(category_folder))
    print(f"Jumlah file pada folder {category}: {file_count}")

TRAIN
Jumlah file pada folder Mass: 1600
Jumlah file pada folder Nodule: 1600
Jumlah file pada folder Normal: 1600
Jumlah file pada folder Pneumonia: 1600
Jumlah file pada folder Tuberculosis: 1600
VALIDATION
Jumlah file pada folder Mass: 200
Jumlah file pada folder Nodule: 200
Jumlah file pada folder Normal: 200
Jumlah file pada folder Pneumonia: 200
Jumlah file pada folder Tuberculosis: 200
TEST
Jumlah file pada folder Mass: 200
Jumlah file pada folder Nodule: 200
Jumlah file pada folder Normal: 200
Jumlah file pada folder Pneumonia: 200
Jumlah file pada folder Tuberculosis: 200
